# Autoencoder Example
Resource: https://medium.com/pytorch/implementing-an-autoencoder-in-pytorch-19baa22647d1

#### To-Do: revise the codes with conv2d. Complete the visualization to compare input images and output images.

What else for fun?


In [2]:
!pip install torch torchvision

In [3]:
import torchvision
import torch.nn as nn
import torch
import torch.optim as optim

In [4]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        # Q: What are the kwargs here? Are they a dictionary of the images?
        # A: kwargs['x'] refers to x argument in the function. 
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        # Q: Why linear? If you use images, then you should use Cov?
        # A: Hmm...the author has vectorized the images.
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed


In [5]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=784).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [6]:
# transform inputs
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(
    root="~/torch_datasets", train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root="~/torch_datasets", train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=4
)

Extracting /home/jtl/torch_datasets/MNIST/raw/train-images-idx3-ubyte.gz to /home/jtl/torch_datasets/MNIST/raw


Extracting /home/jtl/torch_datasets/MNIST/raw/train-labels-idx1-ubyte.gz to /home/jtl/torch_datasets/MNIST/raw


Extracting /home/jtl/torch_datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/jtl/torch_datasets/MNIST/raw


Extracting /home/jtl/torch_datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/jtl/torch_datasets/MNIST/raw
Processing...
Done!


In [7]:
epochs = 20

for epoch in range(epochs):
    # sw: pay attention to how they computed the loss by batches...This is a great way to eval models.    
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        # sw: pay attention to how they computed the loss by batches...This is a great way to eval models.
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))



epoch : 1/20, loss = 0.031227
epoch : 2/20, loss = 0.017237
epoch : 3/20, loss = 0.015231
epoch : 4/20, loss = 0.014320
epoch : 5/20, loss = 0.013769
epoch : 6/20, loss = 0.013344
epoch : 7/20, loss = 0.013033
epoch : 8/20, loss = 0.012761
epoch : 9/20, loss = 0.012534
epoch : 10/20, loss = 0.012350
epoch : 11/20, loss = 0.012183
epoch : 12/20, loss = 0.012062
epoch : 13/20, loss = 0.011946
epoch : 14/20, loss = 0.011846
epoch : 15/20, loss = 0.011768
epoch : 16/20, loss = 0.011682
epoch : 17/20, loss = 0.011619
epoch : 18/20, loss = 0.011559
epoch : 19/20, loss = 0.011516
epoch : 20/20, loss = 0.011458
